In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
import requests
import time

In [2]:
#tartiblanmagan liknlarni olish va takrorlanganlarini olib tashlash.
# Read in data into a dataframe 
data = pd.read_csv('all_kun.csv')
# Display top of dataframe
data.head()
#takrorlangan blok linklarni olib tashladik
data1=data.drop_duplicates(['0'],keep= 'last')
data1.shape

(602, 1)

In [3]:
data.shape

(1200, 1)

In [16]:
#har bir blok likni ichida o'rtacha 27 ta yangilik likni bor va o'sha liknlarni hammasini ajratib olmiz

all_kun_links=[]
for i in data1['0']:
    url = str(i)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    quotes = soup.find('div', class_="daily-news max-w").find_all(href=True)
    for quote in quotes:
        link = "https://kun.uz/uz"+quote['href']
        all_kun_links.append(link)
    
print(len(all_kun_links))

16254


In [17]:
#listni dataframega aylanitib oldik
df = pd.DataFrame(all_kun_links,columns=['link'])
print (df)

                                                    link
0      https://kun.uz/uz/news/2022/06/24/pedagogik-ta...
1      https://kun.uz/uz/news/2022/06/24/ota-onam-oqi...
2      https://kun.uz/uz/news/2022/06/24/ozbekiston-a...
3      https://kun.uz/uz/news/2022/06/24/sogdiyona-qi...
4      https://kun.uz/uz/news/2022/06/24/toshkentda-k...
...                                                  ...
16249  https://kun.uz/uz/news/2021/08/21/iyul-oyida-e...
16250  https://kun.uz/uz/news/2021/08/21/ozbekistonda...
16251  https://kun.uz/uz/news/2021/08/21/bugun-ozbeki...
16252  https://kun.uz/uz/news/2021/08/21/toshkent-vil...
16253  https://kun.uz/uz/news/2021/08/21/tana-harorat...

[16254 rows x 1 columns]


In [18]:
# va bunda ham takrorlangan liknlarni olib tahsladik
df1=df.drop_duplicates(['link'],keep= 'last')
df1.shape

(16253, 1)

In [19]:
#unikalniy 16253 ta liknki saqlab oldik
df1.to_csv("all_uz_kun.csv", index = False)

In [28]:
df1.shape[0]

16253

In [39]:
start = time.time()
data=[]
for i in range(0,df1.shape[0]):
    try:
        sub_data=[]
        response_link = requests.get(df1['link'][i])
        if(response_link.status_code == 200):
            #get all data form page
            soup_link = BeautifulSoup(response_link.text, 'lxml')
            my_txt=''
            try:
                # get question title
                title = soup_link.select_one('div[class*="single-header__title"]').get_text().replace('\n','').replace('\r','').replace('\t','')
                #print("title: ",title)
                my_txt = my_txt+ title
            except Exception as ex:
                pass
            
            try:
                title2 = soup_link.find('h4').text.replace('\n','').replace('\r','').replace('\t','')
                #print("title2: ",title2)
                my_txt = my_txt+ title2
            except Exception as ex:
                pass
            
            
            quotes_text = soup_link.find_all('p')
            for j in range(0,len(quotes_text)):
                line=quotes_text[j].text.replace('\n','').replace('\r','').replace('\t','')
                # print(line,'--end')
                my_txt = my_txt+ ' '+line
                
            sub_data.append(df1['link'][i])
            sub_data.append(my_txt)
            
                
            data.append(sub_data)
                   
        else:
            print(i, "boyicha link topilmadi")
        
    except Exception as ex:
        print("ex ",i,ex)
    
        
end = time.time()
print(end - start)


ex  80 80
4882.983828306198


In [40]:
len(data)

16252

In [41]:
df2 = pd.DataFrame(data,columns=['link','text'])
df2

,link,text
0,https://kun.uz/uz/news/2022/06/24/pedagogik-ta...,Pedagogik ta’lim bo‘yicha qarorlar: nimalar o‘...
1,https://kun.uz/uz/news/2022/06/24/ota-onam-oqi...,“Ota-onam o‘qishga rag‘batni doim birinchi o‘r...
2,https://kun.uz/uz/news/2022/06/24/ozbekiston-a...,O‘zbekiston-AQSh: Hamkorlik va sherikchilik al...
3,https://kun.uz/uz/news/2022/06/24/sogdiyona-qi...,«So‘g‘diyona» Qirg‘iziston klubidan ustun keli...
4,https://kun.uz/uz/news/2022/06/24/toshkentda-k...,Toshkentda kovid-markazlar qayta ochilmoqda Sh...
...,...,...
16247,https://kun.uz/uz/news/2021/08/21/dxsh-loyihal...,DXSh loyihalarini moliyalashtirish tartibi bel...
16248,https://kun.uz/uz/news/2021/08/21/iyul-oyida-e...,Iyul oyida eng ko‘p «Damas» avtomobili ishlab ...
16249,https://kun.uz/uz/news/2021/08/21/ozbekistonda...,O‘zbekistonda 1 kvadrat kilometr yer maydoniga...
16250,https://kun.uz/uz/news/2021/08/21/bugun-ozbeki...,Bugun O‘zbekiston bo‘yicha kam bulutli ob-havo...


In [42]:
#unikalniy 16253 ta liknki saqlab oldik
df2.to_csv("all_uz_kun_data.csv", index = False)

In [43]:
print(1)

1


In [44]:
df2['link'][16251]

'https://kun.uz/uz/news/2021/08/21/toshkent-viloyati-gosht-yetishtirish-boyicha-yetakchi-bolib-turibdi'